In [2]:
# Classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split



In [3]:
import pandas as pd
# /Users/arrao/Documents/Atchut/Book-PPML/Chapter6/simulated_transactions_csv3/transactions.csv 

url ="/Users/arrao/Documents/Atchut/Book-PPML/Chapter6/fraud_transactions.csv"
#url = "/Users/arrao/Documents/Atchut/Book-PPML/Chapter6/simulated_transactions_csv/2023-07-08.csv"
df_actual = pd.read_csv(url, sep=",")
df_actual.head()

,Unnamed: 0,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD
0,0,0,2023-02-01 00:43:37,901,8047,82,1
1,1,1,2023-02-01 01:20:13,2611,7777,15,0
2,2,2,2023-02-01 01:22:52,4212,3336,53,0
3,3,3,2023-02-01 01:26:40,1293,7432,59,0
4,4,4,2023-02-01 01:52:23,2499,1024,25,0


In [4]:
df_transactions = df_actual[['CUSTOMER_ID','TERMINAL_ID','TX_AMOUNT','TX_FRAUD']]
#df_transactions=df_transactions.head(10)
df_transactions

,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD
0,901,8047,82,1
1,2611,7777,15,0
2,4212,3336,53,0
3,1293,7432,59,0
4,2499,1024,25,0
...,...,...,...,...
4557161,1465,7455,92,1
4557162,4009,3429,36,0
4557163,1336,3116,50,0
4557164,1611,3314,81,1


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

print('No Frauds', round(df_transactions['TX_FRAUD'].value_counts()[0]/len(df_transactions) * 100,2), '% of the dataset')
print('Frauds', round(df_transactions['TX_FRAUD'].value_counts()[1]/len(df_transactions) * 100,2), '% of the dataset')

X = df_transactions.drop('TX_FRAUD', axis=1)
y = df_transactions['TX_FRAUD']



No Frauds 75.71 % of the dataset
Frauds 24.29 % of the dataset


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values


In [11]:
# Turn the values into an array for feeding the classification algorithms.
X_train

array([[4548, 8796,   17],
       [1141,   95,   52],
       [4740, 6652,   51],
       ...,
       [4084, 6441,   48],
       [3433, 8781,   34],
       [ 394, 6514,   72]])

In [12]:
X_test

array([[4152,  324,   11],
       [3523, 9137,   51],
       [2321, 4380,   11],
       ...,
       [3666,  225,   11],
       [3173, 4713,   98],
       [1591, 2761,   88]])

In [13]:
from sklearn.model_selection import cross_val_score

logreg = LogisticRegression(random_state=0)

In [14]:
logreg.fit(X_train, y_train)
training_score = cross_val_score(logreg, X_train, y_train, cv=2)
print('Logistic Regression Cross Validation Score: ', round(training_score.mean() * 100, 2).astype(str) + '%')


Logistic Regression Cross Validation Score:  100.0%


In [15]:
import numpy as np
np.sum(logreg.predict(X_test) == y_test)/X_test.shape[0]

1.0

In [16]:
logreg.intercept_[0], logreg.coef_[0]

(-1168.308115256604,
 array([-2.47724513e-05,  3.17749573e-06,  1.54748556e+01]))

In [17]:
logreg.coef_

array([[-2.47724513e-05,  3.17749573e-06,  1.54748556e+01]])

In [18]:
import phe as paillier
pubkey, privkey = paillier.generate_paillier_keypair(n_length=1024)

In [19]:
coef = logreg.coef_[0, :]
encrypted_weights = [pubkey.encrypt(coef[i]) for i in range(coef.shape[0])]
encrypted_intercept =pubkey.encrypt(logreg.intercept_[0])

In [20]:
print(encrypted_weights)

[<phe.paillier.EncryptedNumber object at 0x7ff005a9c9d0>, <phe.paillier.EncryptedNumber object at 0x7fefe3b2c520>, <phe.paillier.EncryptedNumber object at 0x7ff010760cd0>]


In [21]:
encrypted_intercept

In [23]:
print(X_test[0])
print(X_test[0][0])
print(X_test[0][1])
print(X_test[0][2])
print(X.shape[0])

[4152  324   11]
4152
324
11
4557166


In [39]:
#y=np.zeros(X_test.shape[0])
#for i in range(X_test.shape[0]):
    #c1 = X_test[i][0]
    #c2 = X_test[i][1]
    #c3 = X_test[i][2]    
    #score = encrypted_intercept
    #score +=  c1 * encrypted_weights[0] + c2 * encrypted_weights[1] + c3 * encrypted_weights[1]
    #print(score)
    #dscore = privkey.decrypt(score)
    #y[i] = dscore

In [40]:
y=np.zeros(100)
for i in range(1,100):
    c1 = X_test[i][0]
    c2 = X_test[i][1]
    c3 = X_test[i][2]    
    score = encrypted_intercept
   # print(score)
    score +=  c1 * encrypted_weights[0] + c2 * encrypted_weights[1] + c3 * encrypted_weights[2]
    #print(score)
    dscore = privkey.decrypt(score)
    #print(dscore)
    y[i] = dscore

In [41]:
print(y[0],y_test[0])
error = np.mean(np.sign(y) != y_test[1:100])
print(error)

0.0 0
1.0


/var/folders/61/3x6_zr2n5nd67km655rj4nd4r4gmsb/T/ipykernel_74797/4232043888.py:2: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  error = np.mean(np.sign(y) != y_test[1:100])
